## Data dependencies

In [1]:
!sha1sum ../data/lope-gpt-resps.json

26fda18f16fa08b09bc168741a4d769ce230a5bd  ../data/lope-gpt-resps.json


## Load Data

In [2]:
import re
import random
import json
from pathlib import Path
from io import StringIO

import numpy as np

In [3]:
resp_data = json.loads(Path("../data/lope-gpt-resps.json").read_text())

In [4]:
next(iter(resp_data.items()))

('丟棄',
 {'word': '丟棄',
  'char_idx': 0,
  'prompt': '請從CWN中「丟」的詞意裡，找出「丟棄」這個詞中「丟」的詞意？',
  'message': '「丟」在「丟棄」這個詞中的詞意是「丟掉廢棄物」。'})

In [5]:
print(np.quantile([len(x["message"]) for x in resp_data.values()], [.95]))
data = [x for x in resp_data.values() if not x["message"].startswith("[")]
print("longest message:", max(len(x["message"]) for x in data))
data[0]

[205.]
longest message: 367


{'word': '丟棄',
 'char_idx': 0,
 'prompt': '請從CWN中「丟」的詞意裡，找出「丟棄」這個詞中「丟」的詞意？',
 'message': '「丟」在「丟棄」這個詞中的詞意是「丟掉廢棄物」。'}

In [6]:
def_task_prompt_base = "{prompt}{message}"

def build_prompt(fstring, **kwargs):
    var_locations = []
    formatted_string = StringIO()    
    while match:=re.search(r'\{(\w+)\}', fstring):
        var_name = match.group(1)
        var_value = str(kwargs[var_name])  
        
        formatted_string.write(fstring[:match.start()])
        var_start = formatted_string.tell()
        formatted_string.write(var_value)
        var_end = formatted_string.tell()
        var_locations.append((var_name, var_value, var_start, var_end))
        fstring = fstring[match.end():]    
    formatted_string.write(fstring)

    ## check var_locations
    outstr = formatted_string.getvalue()
    for _, val, si, ei in var_locations:
        assert outstr[si:ei] == val
        
    return outstr, var_locations


In [7]:
def shuffle_data(indata, rng):
    """
    Shuffles the first element of each pair in a list of data pairs randomly.

    Args:
        data_pairs (list): A list of pairs, where each pair is a tuple of two elements.

    Returns:
        list: A new list of pairs, where each pair is a tuple of two elements. The first element of each pair
        is shuffled randomly.
    """    
    shuffle_items = [(x["message"], x["word"]) for x in indata]
    rng.shuffle(shuffle_items)
    perm_data = [{**d, "message": s[0], "perm_src": s[1]} for d, s in zip(indata, shuffle_items)]
    return perm_data

In [8]:
rng = random.Random(123)
perm_data = shuffle_data(data, rng)

In [9]:
build_prompt(def_task_prompt_base, prompt=data[0]["prompt"], message=data[0]["message"])

('請從CWN中「丟」的詞意裡，找出「丟棄」這個詞中「丟」的詞意？「丟」在「丟棄」這個詞中的詞意是「丟掉廢棄物」。',
 [('prompt', '請從CWN中「丟」的詞意裡，找出「丟棄」這個詞中「丟」的詞意？', 0, 31),
  ('message', '「丟」在「丟棄」這個詞中的詞意是「丟掉廢棄物」。', 31, 55)])

In [10]:
build_prompt(def_task_prompt_base, prompt=perm_data[0]["prompt"], message=perm_data[0]["message"])

('請從CWN中「丟」的詞意裡，找出「丟棄」這個詞中「丟」的詞意？「設廠」這個詞中的「廠」的詞意有多個，包括：\n\n1. 運用技術將原料製成特定物品的場所。\n2. 工廠的建築物及建築物所在的位置。\n3. 從事製造生產的公司。\n4. 修理保養特定器具的建築物及建築物所在的位置。\n5. 比喻保養特定身體部位的建築物及建築物所在的位置。\n6. 姓。\n\n請問還有其他問題嗎？',
 [('prompt', '請從CWN中「丟」的詞意裡，找出「丟棄」這個詞中「丟」的詞意？', 0, 31),
  ('message',
   '「設廠」這個詞中的「廠」的詞意有多個，包括：\n\n1. 運用技術將原料製成特定物品的場所。\n2. 工廠的建築物及建築物所在的位置。\n3. 從事製造生產的公司。\n4. 修理保養特定器具的建築物及建築物所在的位置。\n5. 比喻保養特定身體部位的建築物及建築物所在的位置。\n6. 姓。\n\n請問還有其他問題嗎？',
   31,
   181)])

## Build pairs

In [11]:
datalist = []
for item_x in data:
    prompt, var_loc = build_prompt(def_task_prompt_base, 
                                   prompt=item_x["prompt"], message=item_x["message"])
    datalist.append({
        "word": item_x["word"],
        "char_idx": item_x["char_idx"],
        "msg_src": item_x["word"],
        "type": "emp",        
        "prompt": prompt,
        "var_loc": var_loc        
    })

for item_x in perm_data:
    prompt, var_loc = build_prompt(def_task_prompt_base, 
                                   prompt=item_x["prompt"], message=item_x["message"])
    datalist.append({
        "word": item_x["word"],
        "char_idx": item_x["char_idx"],
        "msg_src": item_x["perm_src"],
        "type": "perm",        
        "prompt": prompt,
        "var_loc": var_loc        
    })

In [12]:
datalist[:1]

[{'word': '丟棄',
  'char_idx': 0,
  'msg_src': '丟棄',
  'type': 'emp',
  'prompt': '請從CWN中「丟」的詞意裡，找出「丟棄」這個詞中「丟」的詞意？「丟」在「丟棄」這個詞中的詞意是「丟掉廢棄物」。',
  'var_loc': [('prompt', '請從CWN中「丟」的詞意裡，找出「丟棄」這個詞中「丟」的詞意？', 0, 31),
   ('message', '「丟」在「丟棄」這個詞中的詞意是「丟掉廢棄物」。', 31, 55)]}]

In [13]:
datalist[-2]

{'word': '首先',
 'char_idx': 0,
 'msg_src': '展望',
 'type': 'perm',
 'prompt': '請從CWN中「首」的詞意裡，找出「首先」這個詞中「首」的詞意？「展望」這個詞中的「展」的詞意是「有規劃地陳列特定物品或示範特定技術供人觀賞」。',
 'var_loc': [('prompt', '請從CWN中「首」的詞意裡，找出「首先」這個詞中「首」的詞意？', 0, 31),
  ('message', '「展望」這個詞中的「展」的詞意是「有規劃地陳列特定物品或示範特定技術供人觀賞」。', 31, 71)]}

In [14]:
out_path = Path("../data/cwn_lopegpt_dataset.json")
out_path.write_text(json.dumps(datalist))
!sha1sum {str(out_path)}

51476df89ae9674b24c033257752a8a56adc7b99  ../data/cwn_lopegpt_dataset.json
